In [1]:
import os
import numpy as np 
import pandas as pd 

In [3]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/stanford_politeness_stack_furthest.csv

--2022-12-21 10:28:07--  https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/stanford_politeness_stack_furthest.csv
Resolving cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)... 54.231.204.233, 52.217.167.145, 52.216.145.19, ...
Connecting to cleanlab-public.s3.amazonaws.com (cleanlab-public.s3.amazonaws.com)|54.231.204.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228462 (223K) [text/csv]
Saving to: './../../../dataset/stanford_politeness_stack_furthest.csv'

stanford_politeness 100%[===================>] 223.11K   794KB/s    in 0.3s    

2022-12-21 10:28:08 (794 KB/s) - './../../../dataset/stanford_politeness_stack_furthest.csv' saved [228462/228462]



In [4]:
path = "./../../../dataset/stanford_politeness_stack_furthest.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (1636, 5)


,text,meta.Binary,true_label,given_label,true_error
0,What is the definition of `buffer`? Is it a lo...,0,13.0,19.0,0
1,do you want to send the 32bit number in binary...,0,13.0,17.0,0
2,"""Cant see the error"" ? You mean your compiler ...",-1,5.0,14.0,0
3,The nth letter? Is this the Roman version of t...,-1,13.0,15.0,0
4,"By literal value, do you mean the string? Or t...",0,13.0,21.0,0


In [5]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,text,meta.Binary,given_label
0,What is the definition of `buffer`? Is it a lo...,0,19.0
1,do you want to send the 32bit number in binary...,0,17.0


In [6]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [7]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [8]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1636
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1273.38 MB
	Train Data (Original)  Memory Usage: 0.28 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator..

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.761036,0.207100,0.059718,0.019938,1.026099,0.059718,0.019938,1.026099,1,True,2
1,WeightedEnsemble_L2,0.531652,0.223525,0.083506,0.031081,3.879120,0.002687,0.000400,0.031454,2,True,4
2,LightGBM,0.433288,0.214287,0.005204,0.001569,1.355640,0.005204,0.001569,1.355640,1,True,1
3,NeuralNetFastAI,0.338719,0.182770,0.015897,0.009174,1.465927,0.015897,0.009174,1.465927,1,True,3


### Cross Validation fitting

In [9]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1636
Train Data Columns: 2
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1272.42 MB
	Train Data (Original)  Memory Usage: 0.28 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.892954,0.220144,0.055043,0.067186,0.883055,0.055043,0.067186,0.883055,1,True,2
1,WeightedEnsemble_L2,0.518230,0.259586,0.193789,0.113985,10.325938,0.001207,0.000177,0.035193,2,True,4
2,LightGBM_BAG_L1,0.436784,0.236661,0.029931,0.009475,4.530607,0.029931,0.009475,4.530607,1,True,1
3,NeuralNetFastAI_BAG_L1,0.379863,0.238252,0.107608,0.037148,4.877083,0.107608,0.037148,4.877083,1,True,3


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))